In [ ]:
!pip install shap

     |████████████████████████████████| 564 kB 7.5 MB/s 


In [ ]:
from sklearn.preprocessing import QuantileTransformer
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn import linear_model
import lightgbm as lgb

In [ ]:
!gdown 14AK65LfYoj7aSjZOXB9HUzbhqaBaxQi0

Downloading...
From: https://drive.google.com/uc?id=14AK65LfYoj7aSjZOXB9HUzbhqaBaxQi0
To: /content/Matrix_complexity.csv
100% 1.39M/1.39M [00:00<00:00, 131MB/s]


In [ ]:
data_all = pd.read_csv('/content/Matrix_complexity.csv')

In [ ]:
cols = data_all.columns

cols = cols.drop(['film', 'Subj'])

X = data_all.reset_index(drop=True)

y = np.array(X['Labels'])

X = X[cols]

X = X.drop('Labels',axis=1)


In [ ]:
X_test = data_all.loc[(data_all['film']  == 2)].reset_index(drop=True)
X_train = data_all.loc[(data_all['film'] != 2)].reset_index(drop=True)

y_train = np.array(X_train['Labels'])
y_test = np.array(X_test['Labels'])
train_film = X_train['film']
train_subj = X_train['Subj']
test_film = X_test['film']
test_subj = X_test['Subj']

X_train = X_train[cols]
X_test = X_test[cols]

X_train = X_train.drop('Labels',axis=1)
X_test = X_test.drop('Labels',axis=1)

In [ ]:
param = {}
param['metric'] = ['MAPE', 'MAE', 'R2']
train_data = lightgbm.Dataset(data_all)

NameError: ignored

In [ ]:
num_round = 100
bst = lightgbm.train(param, train_data, num_round)

In [ ]:
ypred = bst.predict(X_test)
ypred

In [ ]:
models = ['lightgbm']
metrics = ['MAPE', 'MAE', 'R2']
formes = ['inverted', 'average']
columns = [
    np.array([models[i] for i in range(1) for j in range(3) for k in range(2)]),
    np.array([metrics[j] for i in range(1) for j in range(3) for k in range(2)]),
    np.array([formes[k] for i in range(1) for j in range(3) for k in range(2)]),
]
chans = ['Fz', 'F3', 'F7','C3', 'T7', 'Pz', 'P3','P7', 'O1', 'Oz', 'O2','P4', 'P8', 'Cz','C4', 'T8', 'F4', 'F8']

In [ ]:
import tqdm

result = pd.DataFrame()
grid_res = []
n_films = 8
n_subjects = 32
for column in data_all.columns:
    data_all[column] = data_all[column].fillna(0)


for film in tqdm.tqdm_notebook(range(1, n_films + 1)):

  df = pd.DataFrame(np.random.randn(1, 1 * 3 * 2), columns=columns)
  X_test = data_all.loc[(data_all['film'] == film)].reset_index(drop=True)
  X_train = data_all.loc[(data_all['film'] != film)].reset_index(drop=True)
  X_train = X_train.sample(frac=1)
  X_test = X_test.sample(frac=1)

  y_train = X_train['Labels']
  y_test_inv = np.array(X_test['Labels'])
  train_film = X_train['film']
  train_subj = X_train['Subj']
  test_film = X_test['film']
  test_subj = X_test['Subj']
  
  X_train = X_train.drop(['film', 'Subj', 'Labels'], axis=1)
  X_test = X_test.drop(['film', 'Subj', 'Labels'], axis=1)
  
  qt_train = QuantileTransformer(output_distribution='normal',
                            copy=True)
  y_train = pd.DataFrame(qt_train.fit_transform(np.array(y_train).reshape(-1, 1)))
  qt_test = QuantileTransformer(output_distribution='normal',
                            copy=True)
  y_test = pd.DataFrame(qt_test.fit_transform(np.array(y_test_inv).reshape(-1, 1)))

  lgb_train = lgb.Dataset(X_train, y_train)
  lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
  params = {
      'boosting_type': 'gbdt',
      'objective': 'regression',
      'metric': {'l2', 'l1'},
      'num_leaves': 31,
      'learning_rate': 0.05,
      'feature_fraction': 0.9,
      'bagging_fraction': 0.8,
      'bagging_freq': 5,
      'verbose': 0
  }

  print('Starting training...')
  # train
  gbm = lgb.train(params,
                  lgb_train,
                  num_boost_round=20,
                  valid_sets=lgb_eval,
                  callbacks=[lgb.early_stopping(stopping_rounds=5)])


  y_pred =  gbm.predict(X_test, num_iteration=gbm.best_iteration)
  preds = qt_test.inverse_transform(np.array(y_pred).reshape(-1, 1))
  # df[('lightgbm', 'R2', 'inverted')] = r2_score(preds, y_test_inv)
  df[('lightgbm', 'MAPE', 'inverted')] = mean_absolute_percentage_error(preds, y_test_inv)
  df[('lightgbm', 'MAE', 'inverted')] = mean_absolute_error(preds, y_test_inv)
  
  pred = [[0 for j in range(n_films)] for i in range(n_subjects)]
  num_chans = [[0 for j in range(n_films)] for i in range(n_subjects)]
  labels = [[0 for j in range(n_films)] for i in range(n_subjects)]
  lbl = [[0 for j in range(n_films)] for i in range(n_subjects)]
  channels = [[] for i in range(n_subjects)]
  for i in range(len(preds)):
      pred[np.array(test_subj)[i] - 1][np.array(test_film)[i] - 1] += preds[i]
      num_chans[np.array(test_subj)[i] - 1][np.array(test_film)[i] - 1] += 1
      labels[np.array(test_subj)[i] - 1][np.array(test_film)[i] - 1] += y_test_inv[i]
  labels = np.array(labels).astype(np.float64) / np.array(num_chans).astype(np.float64)
  pred = np.array(pred).astype(np.float64) / np.array(num_chans).astype(np.float64)
  labels = np.nan_to_num(labels)
  pred = np.nan_to_num(pred)
  predicted = []
  lbl = []
  for i in range(len(labels)):
      if sum(labels[i]) != 0:
          for j in range(n_films):
              if labels[i][j] != 0:
                  lbl.append(labels[i][j])  
                  predicted.append(pred[i][j])

  # df[('lightgbm', 'R2', 'average')] = r2_score(lbl, predicted)
  df[('lightgbm', 'MAPE', 'average')] = mean_absolute_percentage_error(lbl, predicted)
  df[('lightgbm', 'MAE', 'average')] = mean_absolute_error(lbl, predicted)
  result = pd.concat([result, df], axis=0)

In [ ]:
result.describe()

lightgbm                                                  
           MAPE                 MAE                  R2          
       inverted   average  inverted   average  inverted   average
count  8.000000  8.000000  8.000000  8.000000  8.000000  8.000000
mean   0.263726  0.519164  1.753069  1.753069 -0.004663  0.433667
std    0.060842  0.213385  0.347841  0.347841  0.521453  0.520626
min    0.205798  0.345098  1.302197  1.302197 -0.663790 -0.289797
25%    0.212793  0.361842  1.470238  1.470238 -0.237151  0.096423
50%    0.260820  0.395873  1.719870  1.719870 -0.166687  0.359981
75%    0.281077  0.696760  2.101942  2.101942  0.093874  0.705609
max    0.387841  0.878067  2.177630  2.177630  1.083310  1.261408